# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

N. Storm  ->  N. Storm  |  ['N. Storm']
M. Bergemann  ->  M. Bergemann  |  ['M. Bergemann']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
G. Rouillé  ->  G. Rouillé  |  ['G. Rouillé']
C. Jäger  ->  C. Jäger  |  ['C. Jäger']
S. Ghosh  ->  S. Ghosh  |  ['S. Ghosh']


Arxiv has 62 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2401.02484


extracting tarball to tmp_2401.02484...

 done.


/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2401.02484/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'nucref' from 'tmp_2401.02484/nucref.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:399: LatexWarning: Latex injecting: 'abundtable' from 'tmp_2401.02484/abundtable.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.18/x64/lib/python3.9/site-packages/arxiv_on_deck_2/

...

extracting tarball to tmp_2401.02821...

 done.
Retrieving document from  https://arxiv.org/e-print/2401.02876


extracting tarball to tmp_2401.02876...

 done.


Found 129 bibliographic references in tmp_2401.02876/CIGs_1Ha-Stars.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.02484-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.02484) | **Spectacular nucleosynthesis from early massive stars**  |
|| A. P. Ji, et al. -- incl., <mark>N. Storm</mark>, <mark>M. Bergemann</mark>, <mark>H.-W. Rix</mark> |
|*Appeared on*| *2024-01-08*|
|*Comments*| *11 pages + 22 page appendix, accepted to ApJL*|
|**Abstract**| Stars formed with initial mass over 50 Msun are very rare today, but they are thought to be more common in the early universe. The fates of those early, metal-poor, massive stars are highly uncertain. Most are expected to directly collapse to black holes, while some may explode as a result of rotationally powered engines or the pair-creation instability. We present the chemical abundances of J0931+0038, a nearby low-mass star identified in early followup of SDSS-V Milky Way Mapper, which preserves the signature of unusual nucleosynthesis from a massive star in the early universe. J0931+0038 has relatively high metallicity ([Fe/H] = -1.76 +/- 0.13) but an extreme odd-even abundance pattern, with some of the lowest known abundance ratios of [N/Fe], [Na/Fe], [K/Fe], [Sc/Fe], and [Ba/Fe]. The implication is that a majority of its metals originated in a single extremely metal-poor nucleosynthetic source. An extensive search through nucleosynthesis predictions finds a clear preference for progenitors with initial mass > 50 Msun, making J0931+0038 one of the first observational constraints on nucleosynthesis in this mass range. However the full abundance pattern is not matched by any models in the literature. J0931+0038 thus presents a challenge for the next generation of nucleosynthesis models and motivates study of high-mass progenitor stars impacted by convection, rotation, jets, and/or binary companions. Though rare, more examples of unusual early nucleosynthesis in metal-poor stars should be found in upcoming large spectroscopic surveys. |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.02876-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.02876) | **Extraplanar emission in isolated edge-on late-type galaxies. I. The  H$α$ distribution versus to the old and young stellar discs**  |
|| M. M. Sardaneta, et al. -- incl., <mark>S. Ghosh</mark> |
|*Appeared on*| *2024-01-08*|
|*Comments*| *Accepted for its publication in MNRAS. Manuscript ID: MN-23-2536-MJ.R1*|
|**Abstract**| Isolated galaxies are the ideal reference sample to study the galaxy structure minimising potential environmental effects. We selected a complete sample of 14 nearby, late-type, highly inclined ($i\geq80^{\circ}$), isolated galaxies from the Catalogue of Isolated Galaxies (CIG) which offers a vertical view of their disc structure. We aim to study extraplanar Diffuse Ionized Gas (eDIG) by comparing the old and young disc components traced by near-infrared (NIR) and Ultraviolet (UV) imaging with the H$\alpha$ emission structure. We obtained H$\alpha$ monochromatic maps from the Fabry-Perot (FP) interferometry, while the old and young discs structures are obtained from the photometric analysis of the 2MASS K$_{s}$-band, and GALEX NUV and FUV images, thereby identifying the stellar disc and whether the eDIG is present. The H$\alpha$ morphology is peculiar in CIG 71, CIG 183, CIG 593 showing clear asymmetries. In general, geometric parameters (isophotal position angle, peak light distribution, inclination) measured from H$\alpha$, UV and NIR show minimal differences (e.g. $\Delta i\leq\pm$10$^{\circ}$), suggesting that interaction does not play a significant role in shaping the morphology, as expected in isolated galaxies. From H$\alpha$ maps, the eDIG was detected vertically in 11 out of 14 galaxies. Although the fraction of eDIG is high, the comparison between our sample and a generic sample of inclined spirals suggests that the phenomenon is uncorrelated to the galaxy environment. As suggested by the extraplanar UV emission found in 13 out of 14 galaxies the star formation extends well beyond the disc defined by the H$\alpha$ map. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.02640-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.02640) | **The Fate of Simple Organics on Titan's Surface: A Theoretical  Perspective**  |
|| X. Yu, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2024-01-08*|
|*Comments*| *11 pages, 4 figures*|
|**Abstract**| Atmospheric photochemistry on Titan continuously transforms methane and nitrogen gases into various organic compounds. This study explores the fate of these molecules when they land on Titan's surface. Our analytical exploration reveals that most simple organics found in Titan's atmosphere, including all nitriles, triple-bonded hydrocarbons, and benzene, land as solids. Only a few compounds are in the liquid phase, while only ethylene remains gaseous. For the simple organics that land as solids, we further examine their interactions with Titan's lake liquids. Utilizing principles of buoyancy, we found that flotation can be achieved via porosity-induced (25-60% porosity) or capillary force-induced buoyancy for HCN ices on ethane-rich lakes. Otherwise, these ices would sink and become lakebed sediments. By evaluating the timescale of flotation, our findings suggest that porosity-induced flotation of millimeter-sized and larger sediments is the only plausible mechanism for floating solids to explain the transient "magic islands" phenomena on Titan's lakes. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2401.02821-b31b1b.svg)](https://arxiv.org/abs/arXiv:2401.02821) | **Water transport through mesoporous amorphous-carbon dust**  |
|| R. Basalgète, <mark>G. Rouillé</mark>, <mark>C. Jäger</mark> |
|*Appeared on*| *2024-01-08*|
|*Comments*| **|
|**Abstract**| The diffusion of water molecules through mesoporous dust of amorphous carbon (a-C) is a key process in the evolution of prestellar, protostellar, and protoplanetary dust, as well as in that of comets. It also plays a role in the formation of planets. Given the absence of data on this process, we experimentally studied the isothermal diffusion of water molecules desorbing from water ice buried at the bottom of a mesoporous layer of aggregated a-C nanoparticles, a material analogous to protostellar and cometary dust. We used infrared spectroscopy to monitor diffusion in low temperature (160 to 170 K) and pressure (6 $\times$ 10$^{-5}$ to 8 $\times$ 10$^{-4}$ Pa) conditions. Fick's first law of diffusion allowed us to derive diffusivity values on the order of 10$^{-2}$ cm$^2$ s$^{-1}$, which we linked to Knudsen diffusion. Water vapor molecular fluxes ranged from 5 $\times$ 10$^{12}$ to 3 $\times$ 10$^{14}$ cm$^{-2}$ s$^{-1}$ for thicknesses of the ice-free porous layer ranging from 60 to 1900 nm. Assimilating the layers of nanoparticles to assemblies of spheres, we attributed to this cosmic dust analog of porosity 0.80-0.90 a geometry correction factor, similar to the tortuosity factor of tubular pore systems, between 0.94 and 2.85. Applying the method to ices and refractory particles of other compositions will provides us with other useful data. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2401.02484.md
    + _build/html/tmp_2401.02484/./abund_Mini_v6.png
    + _build/html/tmp_2401.02484/./feh_xh_withdg_v3.png
    + _build/html/tmp_2401.02484/./best_nucl_models_v3.png
exported in  _build/html/2401.02876.md
    + _build/html/tmp_2401.02876/./figures/maps_cb_2/cig71-cb.png
    + _build/html/tmp_2401.02876/./figures/isolation_3b.png
    + _build/html/tmp_2401.02876/./figures/maps_cb_2/cig95-cb.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\url}[1]{\href{#1}{#1}}$
$\newcommand{\dodoi}[1]{doi:~\href{http://doi.org/#1}{\nolinkurl{#1}}}$
$\newcommand{\doeprint}[1]{\href{http://ascl.net/#1}{\nolinkurl{http://ascl.net/#1}}}$
$\newcommand{\doarXiv}[1]{\href{https://arxiv.org/abs/#1}{\nolinkurl{https://arxiv.org/abs/#1}}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\TheStar}{J0931+0038\xspace}$
$\newcommand{\GaiaID}{\gaia DR3 3841101888330639872}$
$\newcommand{\code}[1]{\texttt{#1}\xspace}$
$\newcommand{\APJ}[1]{\textcolor{orange}{[AJ: #1]}}$
$\newcommand{\gaia}{\textit{Gaia}\xspace}$
$\newcommand{\Gaia}{\gaia}$
$\newcommand{\unit}[1]{\ensuremath{\mathrm{ #1}}\xspace}$
$\newcommand{\feh}{\unit{[Fe/H]}}$
$\newcommand{\teff}{\ensuremath{T_\mathrm{eff}}\xspace}$
$\newcommand{\logg}{\ensuremath{\log g}\xspace}$
$\newcommand{\alphafe}{\unit{[\alpha/Fe]}}$
$\newcommand{\vhel}{\mbox{v_{\mathrm{hel}}}}$
$\newcommand{\sigmav}{\mbox{{\sigma_v}}}$
$\newcommand{\sigmafeh}{\mbox{\sigma_{\mathrm{[Fe/H]}}}}$
$\newcommand{\kms}{\unit{km s^{-1}}}$
$\newcommand{\masyr}{\unit{mas yr^{-1}}}$
$\newcommand{\msun}{\unit{M_\odot}}$
$\newcommand{\lsun}{\unit{L_\odot}}$
$\newcommand{\sanj}[1]{\textsf{\color{cyan}{#1}}}$
$\newcommand{\mb}[1]{\textsf{\color{brown}{#1}}}$
$\newcommand{\minesweeper}{\texttt{MINESweeper}\xspace}$
$\newcommand{\cmark}{\ding{51}}$
$\newcommand{\xmark}{\ding{55}}$
$\newcommand{\cxmark}{\textsf{\textcolor{red}{\xmark}}\xspace}$
$\newcommand{çmark}{\textsf{\textcolor{OliveGreen}{\cmark}}\xspace}$
$\newcommand{\cxmarkq}{\textsf{\textcolor{Dandelion}{\xmark?}}\xspace}$
$\newcommand{çmarkq}{\textsf{\textcolor{YellowGreen}{\cmark?}}\xspace}$
$\newcommand{\}{natexlab}$</div>



<div id="title">

# Spectacular nucleosynthesis from early massive stars

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2401.02484-b31b1b.svg)](https://arxiv.org/abs/2401.02484)<mark>Appeared on: 2024-01-08</mark> -  _11 pages + 22 page appendix, accepted to ApJL_

</div>
<div id="authors">

Alexander~P.~Ji, et al.

</div>
<div id="abstract">

**Abstract:** Stars formed with initial mass over 50 $M_\odot$ are very rare today, but they are thought to be more common in the early universe.The fates of those early, metal-poor, massive stars are highly uncertain. Most are expected to directly collapse to black holes, while some may explode as a result of rotationally powered engines or the pair-creation instability.We present the chemical abundances of J0931+0038, a nearby low-mass star identified in early followup of SDSS-V Milky Way Mapper, which preserves the signature of unusual nucleosynthesis from a massive star in the early universe.J0931+0038 has relatively high metallicity ( [ Fe/H ] $= -1.76 \pm 0.13$ ) but an extreme odd--even abundance pattern, with some of the lowest known abundance ratios of [ N/Fe ] , [ Na/Fe ] , [ K/Fe ] , [ Sc/Fe ] , and [ Ba/Fe ] .The implication is that a majority of its metals originated in a single extremely metal-poor nucleosynthetic source.An extensive search through nucleosynthesis predictions finds a clear preference for progenitors with initial mass $> 50 M_\odot$ , making J0931+0038 one of the first observational constraints on nucleosynthesis in this mass range. However the full abundance pattern is not matched by any models in the literature.J0931+0038 thus presents a challenge for the next generation of nucleosynthesis models and motivates study of high-mass progenitor stars impacted by convection, rotation, jets, and/or binary companions.Though rare, more examples of unusual early nucleosynthesis in metal-poor stars should be found in upcoming large spectroscopic surveys.

</div>

<div id="div_fig1">

<img src="tmp_2401.02484/./abund_Mini_v6.png" alt="Fig13" width="100%"/>

**Figure 13. -** 
    Initial mass vs maximum [Fe/H], [C/O], [Na/Mg], [Mg/Fe], and [Zn/Fe]
    for zero-metallicity supernovae from five yield grids:
    CCSN from \citealt{Heger10}(HW10, fiducial mixing, $E = 0.3-2.4$B in steps of 2x); HN from \citealt{Grimmett2018MNRAS}(G18, no mixing, $E=5,10,50,100$B); PISN from \citealt{Heger2002ApJ}(HW02); all types of SN from \citealt{Nomoto2013}(NKT13), and CCSN from \citealt{Ebinger2020ApJ}(E20).
    Point sizes are proportional to log explosion energy (0.3-100B).
    The horizontal red lines and shaded regions show the abundance of $\TheStar$.
    Top row: maximum [Fe/H] strongly prefers higher mass progenitor stars with $M > 50 $\msun$$. Larger explosion energies tend to synthesize more Fe but dilute into more H, with the balance indicated by vertical trends in point sizes.
    Second row: [C/O] rules out low-mass CCSN ($M \lesssim 20 $\msun$$).
    Third and fourth row: [Na/Mg] and [Mg/Fe] disfavor intermediate-mass CCSN ($20-80 $\msun$$).
    Higher energy HN match the abundances better and allow progenitors down to $40 $\msun$$, but these all strongly violate the [Fe/H] constraint.
    E20 is removed from the [Na/Mg] plot as the Na yields are not predicted.
    Fifth row: PISNe ($M > 140 $\msun$$) are unable to produce significant Zn.
    Bottom row: estimated stellar fate given initial mass for a single, metal-poor, non-rotating star \citep{Heger2003}.
     (*fig:abund_Mini*)

</div>
<div id="div_fig2">

<img src="tmp_2401.02484/./feh_xh_withdg_v3.png" alt="Fig1" width="100%"/>

**Figure 1. -** 
    Composition of $\TheStar$(red square) for Na, Ba, Mg, and Sr compared to the SAGA database (grey points), higher mass classical dwarf galaxies (blue squares), and lower mass ultra-faint dwarf galaxies (yellow diamonds).
    The dashed black line is an outlier-clipped 3rd order polynomial fit to the SAGA data.
    The horizontal red line is the [X/H] of $\TheStar$.
    From the top two panels, the red shaded region indicates $\mbox{[Fe/H]} < -3$, the ISM metallicity range that alone would contribute all of the Na, Ba, and other under-abundant elements like N, K, and Sc observed in $\TheStar$.
    The bottom two panels, Mg and Sr, show that explaining the low N, Na, K, Sc, and Ba by simply adding Fe (e.g. with a Type Ia supernova) would require extremely high abundances of other elements.
     (*fig:fehxh*)

</div>
<div id="div_fig3">

<img src="tmp_2401.02484/./best_nucl_models_v3.png" alt="Fig2" width="100%"/>

**Figure 2. -** 
    Representative best matching nucleosynthesis models.
    Abundances of $\TheStar$ are given as red squares with error bars.
    In the top panel, the large open blue circles connected by a solid line indicate the best-fit HN model \citep{Grimmett2018MNRAS}
    The dotted blue line with small points and dashed blue line with small open circles indicate adding the HN to a high-entropy proton-rich wind pattern \citep{Bliss2018ApJ} and weak r-process pattern \citep{Holmbeck2023}, respectively.
    In the bottom panel, the solid purple circles with solid line indicate the best-fit PISN model \citep{Heger2002ApJ}.
    The dotted purple line with small open circles indicates the same model with a modified i-process (calculated using the framework in \citealt{Roederer2022}, see Appendix \ref{app:nuc} for details).
    The HN is the best fit to the lighter elements and has leeway to fit the heavier elements, but the high energy underpredicts [Fe/H].
    The PISN easily matches the high [Fe/H], and we speculate that a full calculation of a metal-enriched PISN progenitor and explosion with i-process could remedy the disadvantages seen here.
     (*fig:nucl_best*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2401.02484"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Extraplanar emission in isolated edge-on late-type galaxies. $\I.The H$$\alpha$ distribution versus to the old and youngstellar discs.$\thanks{Based on observations obtained at the Observatoire deHaute Provence (OHP, France), operated by the French CNRS.}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2401.02876-b31b1b.svg)](https://arxiv.org/abs/2401.02876)<mark>Appeared on: 2024-01-08</mark> -  _Accepted for its publication in MNRAS. Manuscript ID: MN-23-2536-MJ.R1_

</div>
<div id="authors">

M. M. Sardaneta, et al. -- incl., <mark>S. Ghosh</mark>

</div>
<div id="abstract">

**Abstract:** Isolated galaxies are the ideal reference sample to study the galaxy structure minimising potential environmental effects. We selected a complete sample of 14 nearby, late-type, highly inclined ( $i\geq80^{\circ}$ ), isolated galaxies from the Catalogue of Isolated Galaxies (CIG) which offers a vertical view of their disc structure.We aim to study extraplanar Diffuse Ionized Gas (eDIG) by comparing the old and young disc components traced by near-infrared ( _NIR_ ) and Ultraviolet ( _UV_ )  imaging with the H $\alpha$ emission structure.We obtained H $\alpha$ monochromatic maps from the Fabry-Perot (FP) interferometry, while the old and young discs structures are obtained from the photometric analysis of the 2MASS _K$_{s_$} -band, and GALEX _NUV_ and _FUV_ images, thereby identifying the stellar disc and whether the eDIG is present.The H $\alpha$ morphology is peculiar in CIG 71, CIG 183, CIG 593 showing clear asymmetries.In general, geometric parameters (isophotal position angle, peak light distribution, inclination) measured from H $\alpha$ , _UV_ and _NIR_ show minimal differences (e.g. $\Delta i\leq\pm$ 10 $^{\circ}$ ), suggesting that interaction does not play a significant role in shaping the morphology, as expected in isolated galaxies.From H $\alpha$ maps, the eDIG was detected vertically in 11 out of 14 galaxies.Although the fraction of eDIG is high, the comparison between our sample and a generic sample of inclined spirals suggests that the phenomenon is uncorrelated to the galaxy environment. As suggested by the extraplanar _UV_ emission found in 13 out of 14  galaxies the star formation extends well beyond the disc defined by the H $\alpha$ map.

</div>

<div id="div_fig1">

<img src="tmp_2401.02876/./figures/maps_cb_2/cig71-cb.png" alt="Fig19" width="100%"/>

**Figure 19. -** 
Example of the layout of the graphs and maps of each galaxy. CIG 71 (UGC 1301).
Multiwavelength maps of the galaxy: (a) the H$\alpha$ 	monochromatic map from FP-GHASP data; (b) the 2MASS _K$_{s_$}-band image; (c) the  DSS _R_-band image; (d) and (e) the GALEX _NUV_ and _FUV_ images, respectively.
In panels (f) and (g) the surface brightness profile of the _K$_{s_$}-band and _NUV/FUV_ emissions are plotted, respectively.
In both panels, yellow vertical lines mark the surface brightness level at 1$\sigma$(dotted line) and 3$\sigma$(solid line) of the _K$_{s_$}-band image.
In panel (g), vertical lines indicate the radii corresponding to the isophotes at a surface brightness level at $\mu_{_NUV_,_FUV_}=$ 28 mag arcsec$^{-2}$ for the _NUV_(blue dot-dashed line) and _FUV_(red dashed line) maps, respectively. Additionally, the vertical black loosely-dashed line at $\sim$5 arcsec shows the approximate FWHM of the GALEX point spread function \citep[see e.g.][]{marino-2010}.
The ellipses fitted to these isophotes are superimposed on the corresponding map with their associated colour.
The common elements in in panels (a), (b), (d) and (e) are:
a colour-bar indicating the flux in units of $\mathrm{erg sec^{-1} cm^{-2}}$,
the arrow tracing a scale of  30 arcsec,
a black circle embedded in a box indicating the spatial resolution of the image,
a black ellipse representing the surface brightness level at 3$\sigma$ on the background of the _K$_{s_$}-band image
and, a black cross ($\times$) pointing the location of the _NIR_ band peak light distribution.
At the bottom of panel (f) we show the relationship between the coefficient $B_{4}$ and the radius  using the _K$_{s_$}-band map to determine if the isophotes tend to have `disc-like’ ($B_4 < 0$) or `boxy’ ($B_4>0$) shape (see Section \ref{Sec:stellarDisc}).
Finally, in panel (a), on the H$\alpha$ monochromatic map were overlaid the 1$\sigma$ isophotes on the background of the _NUV_ and _FUV_ images.
 (*Fig:maps-example*)

</div>
<div id="div_fig2">

<img src="tmp_2401.02876/./figures/isolation_3b.png" alt="Fig1" width="100%"/>

**Figure 1. -** Comparison between the local number density ($\eta_{k}$)  and tidal strength ($Q_{k}$) parameters for the CIG galaxies in our sample.  These parameters were obtained first, based on digitized photographic plates from POSS-1 and POSS-2 \citep[][]{verley-2007, verley-2007-i}(red triangles) and later, on available  photometric and spectroscopic  data from SDSS-DR9  ([ and Argudo-Fernández 2013](https://ui.adsabs.harvard.edu/abs/2013A&A...560A...9A)) (blue circles). The CIG catalogue numbers are indicated on top the circles and below the triangles.
According to [ and Verley (2007)](http://adsabs.harvard.edu/abs/2007A%26A...472..121V), the horizontal ($\eta_{k}=2.4$)  and vertical ($Q_{k}=-2$) lines enclose the _ region of bona fide_ isolated galaxies in the AMIGA sample.
 (*Fig:isolation*)

</div>
<div id="div_fig3">

<img src="tmp_2401.02876/./figures/maps_cb_2/cig95-cb.png" alt="Fig4" width="100%"/>

**Figure 4. -** CIG 95 (UGC 1733). Same as Figure \ref{Fig:maps-example}. (*Fig:maps-c95*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2401.02876"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

353  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

6  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
